In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from numpy.core.numeric import False_
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime


import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.models import Sequential

np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
data_dir = "/content/drive/MyDrive/food"

batch_size = 64
imgSize = 224

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir, seed=123, subset="training", validation_split=0.2,
  image_size=(imgSize, imgSize), batch_size=batch_size
)

valid_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir, seed=123, subset="validation", validation_split=0.2,
  image_size=(imgSize, imgSize), batch_size=batch_size
)

classNum = len(train_ds.class_names)
labArr = train_ds.class_names
print(train_ds.class_names)

In [ ]:
# Data expend for image augmentation
expendRound = 1
temp_ds = train_ds
for i in range(expendRound):
    train_ds = train_ds.concatenate(temp_ds)

train_ds_Conbine = tf.data.Dataset.zip((train_ds, train_ds))

In [ ]:
# Cutmix distribution
def sample_beta_distribution(size, concentration_0=0.2, concentration_1=0.2):
    gamma_1_sample = tf.random.gamma(shape=[size], alpha=concentration_1)
    gamma_2_sample = tf.random.gamma(shape=[size], alpha=concentration_0)
    return gamma_1_sample / (gamma_1_sample + gamma_2_sample)

# Cutmix box generator
def get_box(lambda_value):
    cut_rat = tf.math.sqrt(1.0 - lambda_value)

    cut_w = imgSize * cut_rat  # rw
    cut_w = tf.cast(cut_w, tf.int32)

    cut_h = imgSize * cut_rat  # rh
    cut_h = tf.cast(cut_h, tf.int32)

    cut_x = tf.random.uniform((1,), minval=0, maxval=imgSize, dtype=tf.int32)  # rx
    cut_y = tf.random.uniform((1,), minval=0, maxval=imgSize, dtype=tf.int32)  # ry

    boundaryx1 = tf.clip_by_value(cut_x[0] - cut_w // 2, 0, imgSize)
    boundaryy1 = tf.clip_by_value(cut_y[0] - cut_h // 2, 0, imgSize)
    bbx2 = tf.clip_by_value(cut_x[0] + cut_w // 2, 0, imgSize)
    bby2 = tf.clip_by_value(cut_y[0] + cut_h // 2, 0, imgSize)

    target_h = bby2 - boundaryy1
    if target_h == 0:
        target_h += 1

    target_w = bbx2 - boundaryx1
    if target_w == 0:
        target_w += 1

    return boundaryx1, boundaryy1, target_h, target_w

def cutmix(train_ds_one, train_ds_two):
    (image1, label1), (image2, label2) = train_ds_one, train_ds_two

    alpha = [0.25]
    beta = [0.25]

    # Get a sample from the Beta distribution
    lambda_value = sample_beta_distribution(1, alpha, beta)

    # Define Lambda
    lambda_value = lambda_value[0][0]

    # Get the bounding box offsets, heights and widths
    boundaryx1, boundaryy1, target_h, target_w = get_box(lambda_value)

    # Get a patch from the second image (`image2`)
    crop2 = tf.image.crop_to_bounding_box(
        image2, boundaryy1, boundaryx1, target_h, target_w
    )
    # Pad the `image2` patch (`crop2`) with the same offset
    image2 = tf.image.pad_to_bounding_box(
        crop2, boundaryy1, boundaryx1, imgSize, imgSize
    )
    # Get a patch from the first image (`image1`)
    crop1 = tf.image.crop_to_bounding_box(
        image1, boundaryy1, boundaryx1, target_h, target_w
    )
    # Pad the `image1` patch (`crop1`) with the same offset
    img1 = tf.image.pad_to_bounding_box(
        crop1, boundaryy1, boundaryx1, imgSize, imgSize
    )

    # Modify the first image by subtracting the patch from `image1`
    # (before applying the `image2` patch)
    image1 = image1 - img1
    # Add the modified `image1` and `image2`  together to get the CutMix image
    image = image1 + image2

    # Adjust Lambda in accordance to the pixel ration
    lambda_value = 1 - (target_w * target_h) / (imgSize * imgSize)
    lambda_value = tf.cast(lambda_value, tf.float32)

    # Combine the labels of both images
    #label = lambda_value * label1 + (1 - lambda_value) * label2

    return image, label1

In [ ]:
# Apply cutmax images to train_ds_cmu
train_ds_cmu = train_ds_Conbine.map(cutmix)

data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
    layers.experimental.preprocessing.RandomContrast(0.1),
  ]
)
# Enable below if doing data_augmentation
#train_ds_cmu = train_ds_cmu.map(lambda image,label:(data_augmentation(image),label))

In [ ]:
# plot images
plt.figure(figsize=(10, 10))
for images, labels in train_ds_cmu.take(1):
  #print(labels)
  labelArr = labels.numpy()
  for i in range(12):
    ax = plt.subplot(3, 4, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title( labArr[labelArr[i]] )
    plt.axis("off")

plt.show()

In [ ]:
baseModel = tf.keras.applications.MobileNetV3Large(input_shape=(imgSize,imgSize,3),
                          include_top=False,
                          weights='imagenet')

baseModel.trainable = True
print("Layers count", len(baseModel.layers))

# fine tune numbers
fine_tune_at = int( len(baseModel.layers) * 0.6)
for layer in baseModel.layers[:fine_tune_at]:
  layer.trainable = False

model = Sequential([
  baseModel,
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(classNum, activation=tf.nn.softmax)
])

In [ ]:
epochsRound = 12
base_learning_rate = 0.0001

checkpoint_filepath = './tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True
)

model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_ds_cmu, epochs=epochsRound,
          validation_data=valid_ds, callbacks=[model_checkpoint_callback])

now = datetime.now()
current_time = now.strftime("%b-%d-%Y_%H:%M:%S")
srt = "/content/drive/MyDrive/savedModel/" + "efficientnetV1B0" + current_time + ".h5"

model.load_weights(checkpoint_filepath)

test_loss, test_acc = model.evaluate(valid_ds, verbose=2)
print(test_acc)

model.save(srt)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochsRound)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# References  

1. CutMix data augmentation for image classification:  
https://keras.io/examples/vision/cutmix/  
2. Get started with TensorBoard  
https://www.tensorflow.org/tensorboard/get_started?hl=zh-tw  
3. Transfer learning and fine-tuning  
https://www.tensorflow.org/tutorials/images/transfer_learning?hl=zh-tw